## Imports

In [99]:
!pip install tabulate
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import from_networkx
from tabulate import tabulate

## Graph Generation Functions

In [100]:
def find_connected_subgraph(G, size=4):
    """Find a connected subgraph of specified size"""
    for component in nx.connected_components(G):
        subgraph = G.subgraph(component)
        if len(subgraph) >= size:
            start_node = np.random.choice(list(subgraph.nodes()))
            nodes = list(nx.bfs_tree(subgraph, start_node))[:size]
            return nodes
    return None

def generate_graph(num_nodes=100, edge_prob=0.05):
    """Generate a random graph ensuring it has at least one connected component of size 4"""
    while True:
        G = nx.erdos_renyi_graph(n=num_nodes, p=edge_prob)
        connected_nodes = find_connected_subgraph(G, size=4)
        if connected_nodes is not None:
            return G, connected_nodes

## Feature Computation Functions

In [101]:
def compute_features(G, nodes):
    """Compute graph features including specific node features"""
    if nodes is None or len(nodes) != 4:
        raise ValueError("Must provide exactly 4 nodes for feature computation")
    
    num_nodes = G.number_of_nodes()
    if num_nodes == 0:
        return torch.zeros(10, dtype=torch.float32)
    
    features = []
    
    # Node-specific features for the provided nodes
    for node in nodes:
        # Basic metrics
        degree = G.degree[node]
        clustering = nx.clustering(G, node)
        avg_neighbor_degree = np.mean([G.degree[n] 
                               for n in G.neighbors(node)]) if list(G.neighbors(node)) else 0
            
        # Centrality metrics
        betweenness = nx.betweenness_centrality(G)[node]
        closeness = nx.closeness_centrality(G)[node]
        pagerank = nx.pagerank(G)[node]
            
        # Handle eigenvector centrality
        try:
            eigenvector = nx.eigenvector_centrality_numpy(G)[node]
        except (nx.NetworkXError, nx.AmbiguousSolution):
            eigenvector = 0
            
        # Structural metrics
        core_number = nx.core_number(G)[node]
        local_efficiency = nx.local_efficiency(G)
            
        node_features = [
            degree,
            clustering,
            avg_neighbor_degree,
            betweenness,
            closeness,
            pagerank,
            eigenvector,
            core_number,
            local_efficiency
        ]
        features.extend(node_features)
    
    # Global node-level features (averaged)
    degrees = [d for _, d in G.degree()]
    clustering_coeffs = [nx.clustering(G, node) for node in G.nodes()]
    neighbor_degrees = [np.mean([G.degree[n] for n in G.neighbors(node)]) if list(G.neighbors(node)) else 0 
                       for node in G.nodes()]
    betweenness = list(nx.betweenness_centrality(G).values())
    closeness = list(nx.closeness_centrality(G).values())
    pagerank = list(nx.pagerank(G).values())
    
    # Handle eigenvector centrality for all nodes
    try:
        eigenvector = list(nx.eigenvector_centrality_numpy(G).values())
    except (nx.NetworkXError, nx.AmbiguousSolution):
        eigenvector = [0] * num_nodes
    
    core_numbers = list(nx.core_number(G).values())
    
    # Calculate global averages
    avg_features = [
        np.mean(degrees),
        np.mean(clustering_coeffs),
        np.mean(neighbor_degrees),
        np.mean(betweenness),
        np.mean(closeness),
        np.mean(pagerank),
        np.mean(eigenvector),
        np.mean(core_numbers),
        nx.local_efficiency(G)
    ]
    
    # Global features
    global_features = [
        nx.density(G)
    ]
    
    # Combine all features
    features.extend(avg_features + global_features)
    return torch.tensor(features, dtype=torch.float32)

def prepare_node_features(G):
    """Prepare node features including removal flag"""
    num_nodes = G.number_of_nodes()
    # Basic features for each node (5 base features + 1 removal flag)
    features = torch.zeros(num_nodes, 6)
    
    for i in range(num_nodes):
        features[i] = torch.tensor([
            G.degree[i],
            nx.clustering(G, i),
            np.mean([G.degree[n] for n in G.neighbors(i)]) if list(G.neighbors(i)) else 0,
            list(nx.betweenness_centrality(G).values())[i],
            list(nx.closeness_centrality(G).values())[i],
            0  # Removal flag, will be set later
        ])
    return features

def prepare_edge_index(G):
    """Convert NetworkX graph edges to PyG edge index"""
    return torch.tensor([[e[0], e[1]] for e in G.edges()]).t().contiguous()

## Data Processing Functions

In [102]:
def process_graph_data(G, nodes_to_remove):
    """Process a graph to create training data"""
    # Original graph features
    original_features = compute_features(G, nodes_to_remove)
    
    # Create residual graph
    residual_G = G.copy()
    residual_G.remove_nodes_from(nodes_to_remove)
    
    # Get features for residual graph
    largest_component = max(nx.connected_components(residual_G), key=len)
    residual_G_main = residual_G.subgraph(largest_component)
    residual_features = compute_features(residual_G_main, 
                                       list(residual_G_main.nodes())[:4] if len(residual_G_main) >= 4 else None)
    
    # Create PyG data object
    data = Data(
        x=prepare_node_features(G),
        edge_index=prepare_edge_index(G),
        removed_nodes=torch.tensor(nodes_to_remove),
        original_features=original_features,
        residual_features=residual_features
    )
    return data

def evaluate_predictions(model, data):
    """
    Evaluate model predictions and print results
    """
    model.eval()
    with torch.no_grad():
        output = model(data)
        loss = compute_loss(output, data)
        accuracy = calculate_accuracy(output, data)
        
        print(f"Evaluation Results:")
        print(f"Loss: {loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
    return output, loss, accuracy

def compute_loss(output, data):
    """Compute MSE loss between predicted and actual features"""
    predicted_features = output.squeeze(0)  # Shape should be [46]
    target_features = data.original_features  # Shape should be [46]
    
    if predicted_features.shape != target_features.shape:
        raise ValueError(f"Shape mismatch: predicted {predicted_features.shape} vs target {target_features.shape}")
    
    return F.mse_loss(predicted_features, target_features)

## Model Definition

In [103]:
class EnhancedGNNModel(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim):
        super(EnhancedGNNModel, self).__init__()
        self.conv1 = GCNConv(node_feature_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        
        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 9)  # 9 features per node
        )
        
        self.global_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 10)  # Global graph features
        )
        
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = data.batch if hasattr(data, 'batch') else torch.zeros(x.size(0), dtype=torch.long)
        
        # Graph convolutions
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv3(x, edge_index).relu()
        
        # Process features for the 4 selected nodes (9 features each = 36)
        node_features = self.node_mlp(x)
        selected_nodes = data.removed_nodes
        selected_features = node_features[selected_nodes]
        selected_features = selected_features.view(-1)  # Flatten to 36 features
        
        # Global features (10 features)
        global_x = global_mean_pool(x, batch)
        global_features = self.global_mlp(global_x).squeeze(0)
        
        # Combine to match target shape (36 + 10 = 46 features)
        combined_features = torch.cat([selected_features, global_features])
        return combined_features.unsqueeze(0)  # Add batch dimension

In [104]:
# Define feature names to match the 46 output features
FEATURE_NAMES = [
    # Node-specific features (9 features for each of 4 nodes = 36)
    *[f"Node{i+1}_{metric}" for i in range(4) for metric in [
        'Degree', 'Clustering', 'NeighborDeg', 'Betweenness', 
        'Closeness', 'PageRank', 'CoreNumber', 'LocalEff', 'Eigenvector'
    ]],
    
    # Global features (10)
    'GraphDensity',
    'AvgClustering',
    'AvgPathLength',
    'DegreeAssortativity',
    'Transitivity',
    'ConnectedComponents',
    'MaxDegree',
    'MinDegree',
    'AvgDegree',
    'GlobalEfficiency'
]

def train_model(model, data, num_epochs=100):
    """Train the model and track feature-specific accuracies"""
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    feature_accuracies = []
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        output = model(data)
        loss = compute_loss(output, data)
        
        # Calculate accuracy for each feature
        accuracies = calculate_feature_accuracy(
            output.squeeze(0), 
            data.original_features
        )
        feature_accuracies.append(accuracies.detach().cpu())
        
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}: Loss = {loss.item():.4f}')
    
    # Average accuracies across epochs
    return torch.stack(feature_accuracies).mean(dim=0)

## Execution

In [105]:
# Generate graph and prepare data
G, selected_nodes = generate_graph(num_nodes=100, edge_prob=0.05)
data = process_graph_data(G, selected_nodes)

# Initialize and train model
model = EnhancedGNNModel(node_feature_dim=6, hidden_dim=64)
avg_accuracies = train_model(model, data)

assert len(FEATURE_NAMES) == len(avg_accuracies), \
    f"Feature names ({len(FEATURE_NAMES)}) and accuracies ({len(avg_accuracies)}) must have same length"

# Create results table
results_dict = {
    'Feature': FEATURE_NAMES,
    'Accuracy': [f"{acc:.4f}" for acc in avg_accuracies]
}
results_df = pd.DataFrame(results_dict)
results_df = results_df.sort_values('Accuracy', ascending=False)

# Display results with nice formatting
print("\nFeature Prediction Accuracies:")
print(tabulate(results_df, headers='keys', tablefmt='pipe', showindex=False))

Epoch 0: Loss = 9.3241
Epoch 10: Loss = 0.8501
Epoch 20: Loss = 0.9213
Epoch 30: Loss = 1.2720
Epoch 40: Loss = 0.6114
Epoch 50: Loss = 0.3766
Epoch 60: Loss = 0.2356
Epoch 70: Loss = 0.5083
Epoch 80: Loss = 0.3085
Epoch 90: Loss = 0.2549

Feature Prediction Accuracies:
| Feature             |   Accuracy |
|:--------------------|-----------:|
| AvgPathLength       |       0.64 |
| GraphDensity        |       0.62 |
| MinDegree           |       0.56 |
| Node3_Degree        |       0.5  |
| Node3_NeighborDeg   |       0.49 |
| Node4_Degree        |       0.47 |
| Node1_LocalEff      |       0.38 |
| Node3_LocalEff      |       0.37 |
| Node2_NeighborDeg   |       0.29 |
| Node4_LocalEff      |       0.29 |
| Transitivity        |       0.23 |
| Node1_Closeness     |       0.2  |
| Node2_Closeness     |       0.2  |
| Node3_Closeness     |       0.19 |
| Node2_LocalEff      |       0.19 |
| Node4_NeighborDeg   |       0.16 |
| Node3_CoreNumber    |       0.15 |
| Node4_CoreNumber    |   